In [ ]:
!python -m spacy download en_core_web_lg

In [11]:
# Contributed to this work on https://towardsdatascience.com/twitter-topic-modeling-e0e3315b12e2
# This file will run LDA for topic modeling with the prediction label for covid data

import pandas as pd
import numpy as np
import json
import requests
import emoji
import regex
import re
import string
from collections import Counter
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls
import spacy
import gensim
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

In [15]:
# read covid prediciton and split into 2 dataset
df = pd.read_csv('covid_predict.csv')
df_nonrummor = df[df['Predicted'] == 0]
df_rummor = df[df['Predicted'] == 1]

# create list of rummor and non-rummor text
rummors = df_rummor['text'].to_numpy()
non_rummors = df_nonrummor['text'].to_numpy()

## Topic Classification for Rummor Tweet

In [61]:
# remove stopwords
from gensim.parsing.preprocessing import remove_stopwords

import nltk
nltk.download('punkt')
rummor_tokens = []
for rummor in rummors:
    rummor = remove_stopwords(rummor)
    token = nltk.word_tokenize(rummor)
    rummor_tokens.append(token)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [62]:
# lemmanizztion
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
for tokens in rummor_tokens:
    for i in range(len(tokens)):
        tokens[i] = lemmatizer.lemmatize(tokens[i])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [96]:
# remove word have length less than 3 character and duplicated word
rummor_token_final =[]
for tokens in rummor_tokens:
    temp=[]
    for token in tokens:
        if len(token) >=3 and token not in temp:
            temp.append(token)
    rummor_token_final.append(temp) 

In [100]:
# Create a id2word dictionary
id2word = Dictionary(rummor_token_final)
print(len(id2word))
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in rummor_token_final]

15596


In [102]:
# Instantiating a Base LDA model 
base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [106]:
# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=rummor_token_final, 
                                   dictionary=id2word, coherence='c_v')
coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -8.645044225328508

Coherence Score:  0.7692178397123337


In [108]:
import pyLDAvis.gensim_models
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(base_model, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.008732 -0.003166       1        1  41.074165
2     -0.017890  0.003430       2        1  25.083305
4     -0.010865 -0.031254       3        1  20.580052
0     -0.014219  0.028945       4        1   9.608630
1      0.051706  0.002045       5        1   3.653848, topic_info=            Term         Freq        Total Category  logprob  loglift
6          covid  1425.000000  1425.000000  Default  30.0000  30.0000
45   coronavirus  1386.000000  1386.000000  Default  29.0000  29.0000
229        trump  1214.000000  1214.000000  Default  28.0000  28.0000
136         like   763.000000   763.000000  Default  27.0000  27.0000
13        people   928.000000   928.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
41         china    18.687329   520.633048   Topic5  -6.2549  -0.0178
552        going    18.286446   512.701286   Topic5  -6.2766  -0.0241
146      million    17.110023   434.532611   Topic5  -6.3431   0.0748
196        right    17.280545   458.995869   Topic5  -6.3331   0.0299
207        state    17.144799   507.104133   Topic5  -6.3410  -0.0776

[472 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
253       1  0.345975  administration
253       2  0.299845  administration
253       3  0.220765  administration
253       4  0.108735  administration
253       5  0.029655  administration
...     ...       ...             ...
1031      2  0.213947            year
1031      3  0.258301            year
1031      4  0.117410            year
1031      5  0.054791            year
1177      4  0.617965             zoo

[950 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 5, 1, 2])

In [118]:
# Hyperparameter tunning with GridSearch
text = [' '.join(map(str, l)) for l in rummor_token_final]

vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(text)


# Define Search Param
search_params = {'n_components': [10, 20, 30], 'learning_decay': [.5]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)
GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128, 
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1, 
                                                 learning_decay=0.7, 
                                                 learning_method=None,
                                                 learning_offset=10.0, 
                                                 max_doc_update_iter=100, 
                                                 max_iter=10,
                                                 mean_change_tol=0.001, 
                                                 n_components=10, 
                                                 n_jobs=1,
                                                 perp_tol=0.1, 
                                                 random_state=None,
                                                 topic_word_prior=None, 
                                                 total_samples=1000000.0, 
                                                 verbose=0),
             n_jobs=1,
             param_grid={'n_topics': [10, 20, 30], 
                         'learning_decay': [0.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
             scoring=None, verbose=0)
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 10}
Best Log Likelihood Score:  -796746.8858647719
Model Perplexity:  14251.956980793346


In [119]:
model_final = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=10,
                       decay=0.5)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [126]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_final, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.019932  0.009805       1        1  31.995645
0      0.009058 -0.001787       2        1  17.307237
5      0.001208  0.017628       3        1  17.250087
3      0.027580 -0.015979       4        1   8.061163
4     -0.012175  0.004822       5        1   7.179506
6      0.008823 -0.010239       6        1   5.007826
8     -0.042310 -0.020225       7        1   4.238913
2     -0.009457  0.032517       8        1   4.001133
1     -0.011197  0.000900       9        1   3.284944
9      0.008537 -0.017443      10        1   1.673546, topic_info=            Term         Freq        Total Category  logprob  loglift
6          covid  1673.000000  1673.000000  Default  30.0000  30.0000
45   coronavirus  1733.000000  1733.000000  Default  29.0000  29.0000
229        trump  1506.000000  1506.000000  Default  28.0000  28.0000
13        people  1016.000000  1016.000000  Default  27.0000  27.0000
173    president   837.000000   837.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
26      american     9.110683   815.400863  Topic10  -6.1924  -0.4040
727         want     8.525918   603.535813  Topic10  -6.2588  -0.1695
27           amp     8.886521   920.983255  Topic10  -6.2174  -0.5507
146      million     7.860033   376.175490  Topic10  -6.3401   0.2220
199         said     7.917199   552.789238  Topic10  -6.3329  -0.1557

[742 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
4017      2  0.075816  abdicating
4017      4  0.682345  abdicating
4017      6  0.151632  abdicating
9046      1  0.843074    absorbed
9046      2  0.064852    absorbed
...     ...       ...         ...
1031      6  0.053512        year
1031      7  0.020067        year
1031      8  0.046823        year
1031      9  0.036790        year
1031     10  0.013378        year

[2898 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 6, 4, 5, 7, 9, 3, 2, 10])

## Topic Classification for Non-Rummor 

In [20]:
from gensim.parsing.preprocessing import remove_stopwords

import nltk
nltk.download('punkt')
non_rummor_tokens = []
for non_rummor in non_rummors:
    non_rummor = remove_stopwords(non_rummor)
    token = nltk.word_tokenize(non_rummor)
    non_rummor_tokens.append(token)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [21]:
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()
nltk.download('wordnet')
for tokens in non_rummor_tokens:
    for i in range(len(tokens)):
        tokens[i] = lemmatizer.lemmatize(tokens[i])

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [22]:
non_rummor_token_final =[]
for tokens in non_rummor_tokens:
    temp=[]
    for token in tokens:
        if len(token) >=3 and token not in temp:
            temp.append(token)
        non_rummor_token_final.append(temp) 

In [23]:
# Create a id2word dictionary
id2word = Dictionary(non_rummor_token_final)
print(len(id2word))
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in non_rummor_token_final]

167618
41331


In [24]:
model_final = LdaMulticore(corpus=corpus,
                       id2word=id2word,
                       num_topics=10,
                       decay=0.5)

In [28]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model_final, corpus, id2word)

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.052599 -0.006946       1        1  16.181789
7     -0.001208  0.002108       2        1  14.513391
2     -0.034412  0.017891       3        1  14.349956
1      0.000587  0.003412       4        1  13.750757
5     -0.018240 -0.001025       5        1  10.064805
9     -0.007965 -0.007230       6        1   8.242822
8      0.047262  0.006088       7        1   7.664742
3     -0.037230 -0.038327       8        1   5.524274
4      0.014959 -0.013301       9        1   4.986322
6     -0.016352  0.037329      10        1   4.721142, topic_info=            Term          Freq         Total Category  logprob  loglift
4          covid  10257.000000  10257.000000  Default  30.0000  30.0000
26   coronavirus   9739.000000   9739.000000  Default  29.0000  29.0000
85          case   3718.000000   3718.000000  Default  28.0000  28.0000
12           new   3245.000000   3245.000000  Default  27.0000  27.0000
27         death   3732.000000   3732.000000  Default  26.0000  26.0000
..           ...           ...           ...      ...      ...      ...
752      million    101.767478   1791.740123  Topic10  -6.3558   0.1849
453      country    115.197498   2909.653035  Topic10  -6.2318  -0.1760
225         week    104.987993   2123.215871  Topic10  -6.3247   0.0463
15      pandemic    113.673610   3325.678525  Topic10  -6.2452  -0.3230
588       health    107.822676   2523.680590  Topic10  -6.2980  -0.0999

[910 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
32125      8  0.849776  _hospital
2610       3  0.888449        _mp
20831     10  0.810210   _samanta
9969       2  0.029350       abba
9969       3  0.675046       abba
...      ...       ...        ...
4270       5  0.176549       zuma
4270       7  0.088274       zuma
4270      10  0.706194       zuma
22791      1  0.936762        โคว
22791      3  0.052042        โคว

[2674 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 3, 2, 6, 10, 9, 4, 5, 7])